In [25]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split, cross_val_score, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler

In [26]:
# Load the data
data = pd.read_csv('data.csv')

In [27]:
# Feature engineering - create an afforestation suitability score based on domain knowledge
data['afforestation_score'] = (data['Average Annual Rainfall (inches)'] * 0.3 + data['Soil Suitability (0 to 1)'] * 0.4 + data['Wildlife Benefit Potential (0 to 1)'] * 0.2 - (data['Population'] / 100000) * 0.1            )

In [28]:
print(data["afforestation_score"].describe())  # Before normalization

count    264.000000
mean      11.093956
std        4.989297
min        0.678681
25%        7.002413
50%       12.240419
75%       14.184746
max       37.565814
Name: afforestation_score, dtype: float64


In [29]:
# Define a reasonable raw score threshold based on domain knowledge
raw_threshold = 11  # Adjust this based on your data
data["good_for_afforestation"] = (data["afforestation_score"] > raw_threshold).astype(int)

In [30]:
# Select features for modeling
features = ['Average Annual Rainfall (inches)', 'Soil Suitability (0 to 1)',
           'Wildlife Benefit Potential (0 to 1)', 'Population']

In [31]:
X = data[features]
y = data['good_for_afforestation']

X_temp, X_test, y_temp, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.15, random_state=42
)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
X_test_scaled = scaler.transform(X_test)

In [32]:
# Set up cross-validation
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

In [33]:
for i in range(0,61):
  print(y[i])

1
1
1
1
0
0
1
1
0
1
0
0
0
0
0
0
1
1
1
1
1
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [34]:
# Define base XGBoost model
base_model = xgb.XGBClassifier(
    objective="binary:logistic",
    random_state=42
)

In [35]:
# Perform cross-validation
cv_scores = cross_val_score(base_model, X_train_scaled, y_train, cv=kfold, scoring='accuracy')
print("\nCross-validation scores:", cv_scores)
print(f"Mean CV accuracy: {cv_scores.mean():.4f}")
print(f"CV standard deviation: {cv_scores.std():.4f}")


Cross-validation scores: [1.         1.         0.97222222 0.97222222 0.97142857]
Mean CV accuracy: 0.9832
CV standard deviation: 0.0137


In [36]:
param_grid = {
    'max_depth': [3, 4, 5],
    'learning_rate': [0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 150],
    'subsample': [0.8, 0.9, 1.0]
}

In [37]:
grid_search = GridSearchCV(
    estimator=base_model,
    param_grid=param_grid,
    cv=kfold,
    scoring='accuracy',
    verbose=1
)

grid_search.fit(X_train_scaled, y_train)
print("\nBest parameters:", grid_search.best_params_)
print(f"Best cross-validation score: {grid_search.best_score_:.4f}")

Fitting 5 folds for each of 81 candidates, totalling 405 fits

Best parameters: {'learning_rate': 0.05, 'max_depth': 3, 'n_estimators': 100, 'subsample': 0.8}
Best cross-validation score: 0.9832


In [38]:
# Train the final model with the best parameters on the combined training and validation sets
best_model = grid_search.best_estimator_
X_train_val = np.vstack((X_train_scaled, X_val_scaled))
y_train_val = pd.concat([y_train, y_val])

best_model.fit(X_train_val, y_train_val)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.05, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [39]:
# Validate on the validation set before final testing
val_predictions = best_model.predict(X_val_scaled)
print("\nValidation Set Results:")
print(f"Accuracy: {accuracy_score(y_val, val_predictions):.4f}")
print("\nValidation Classification Report:")
print(classification_report(y_val, val_predictions))


Validation Set Results:
Accuracy: 1.0000

Validation Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        16
           1       1.00      1.00      1.00        16

    accuracy                           1.00        32
   macro avg       1.00      1.00      1.00        32
weighted avg       1.00      1.00      1.00        32



In [40]:
# Final evaluation on the test set
test_predictions = best_model.predict(X_test_scaled)
print("\nTest Set Results:")
print(f"Accuracy: {accuracy_score(y_test, test_predictions):.4f}")
print("\nTest Classification Report:")
print(classification_report(y_test, test_predictions))


Test Set Results:
Accuracy: 0.9811

Test Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.93      0.97        15
           1       0.97      1.00      0.99        38

    accuracy                           0.98        53
   macro avg       0.99      0.97      0.98        53
weighted avg       0.98      0.98      0.98        53



In [19]:
# Function to get afforestation suitability by state
def get_afforestation_locations(state, model, features, scaler):
    """
    Input: State name
    Output: List of suitable locations for afforestation in the specified state.
    """
    state_data = data[data["State"] == state].copy()

    if state_data.empty:
        return f"No data available for {state}."

    # Prepare features for prediction
    X_state = state_data[features]
    X_state_scaled = scaler.transform(X_state)

    # Predict suitability
    predictions = model.predict(X_state_scaled)
    probabilities = model.predict_proba(X_state_scaled)[:, 1]  # Probability of class 1

    # Add predictions to the state data
    state_data["Prediction"] = predictions
    state_data["Probability"] = probabilities

    # Filter for good locations (Prediction == 1)
    good_locations = state_data[state_data["Prediction"] == 1]

    if good_locations.empty:
        return f"No suitable locations found for afforestation in {state}."

    # Return only the location names
    return good_locations[["City", "Probability"]].sort_values(by="Probability", ascending=False)



In [20]:
# Basic Input-Output system
def main():
    state_input = input("Enter the state you want to check for afforestation suitability: ")
    result = get_afforestation_locations(state_input, best_model, features, scaler)

    if isinstance(result, str):  # If the result is a message (e.g., "No data available")
        print(result)
    else:
        print(f"Suitable locations for afforestation in {state_input}:")
        for index, row in result.iterrows():
            print(f"- {row['City']} (Probability: {row['Probability']:.4f})")


In [21]:
def predict_afforestation_suitability(model):
    # Create a feature array for the new location
    rainfall = float(input("Enter rainfall in inches"))
    soil_suitability = float(input("Enter soil suitability (0 to 1)"))
    wildlife_potential = float(input("Enter wildlife potential (0 to 1)"))
    population = float(input("Enter population"))
    new_location = np.array(
        [[rainfall, soil_suitability, wildlife_potential, population]]
    )

    # Scale the features
    new_location_scaled = scaler.transform(new_location)

    # Make prediction
    prediction = model.predict(new_location_scaled)[0]
    probability = model.predict_proba(new_location_scaled)[0][1]

    if prediction == 1:
        suitability = "Good"
    else:
        suitability = "Not Good"

    return suitability, probability


In [23]:
main()

Enter the state you want to check for afforestation suitability: Texas
Suitable locations for afforestation in Texas:
- Houston (Probability: 0.9816)


In [41]:
predict_afforestation_suitability(best_model)

Enter rainfall in inches40
Enter soil suitability (0 to 1)0.3
Enter wildlife potential (0 to 1)0.5
Enter population657483


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


('Good', 0.7840042)